In [1]:
from utils.tools import dotdict

args = dotdict()
args.target = 'OT'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.gpu = 0
args.lradj = 'type1'
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False
args.freq = 'h'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.is_trainging = True
args.root_path = './data/ETT'
args.data_path ='ETTh1.csv'
args.model_id='ETTh1_96_24'
args.model = 'Autoformer'
args.data = 'ETTh1'
args.features = 'M'
args.seq_len = 96
args.label_len = 48
args.pred_len = 24
args.e_layers = 2
args.d_layers = 1
args.n_heads = 8
args.factor = 1
args.enc_in = 7
args.dec_in =7
args.c_out = 7
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 1
args.distil = True
args.output_attention = False
args.patience= 3
args.learning_rate = 0.0001
args.batch_size = 32
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'
args.train_epochs = 10

## Training

In [2]:

from exp.exp_main import Exp_Main
exp = Exp_Main(args) # set experiments

Use GPU: cuda:0


In [3]:
import torch

setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 1)

print(1)
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
exp.train(setting)#setting是用来保存模型的名字用的，很细节
print(2)
print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
exp.test(setting)
torch.cuda.empty_cache()
print(3)

1
>>>>>>>start training : ETTh1_96_24_Autoformer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_1>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.4203987
	speed: 0.0841s/iter; left time: 215.3882s
	iters: 200, epoch: 1 | loss: 0.3379647
	speed: 0.0767s/iter; left time: 188.8249s
Epoch: 1 cost time: 21.21646213531494
Epoch: 1, Steps: 266 | Train Loss: 0.3549941 Vali Loss: 0.6255439 Test Loss: 0.3930913
Validation loss decreased (inf --> 0.625544).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.3257124
	speed: 0.1952s/iter; left time: 448.0773s
	iters: 200, epoch: 2 | loss: 0.2563295
	speed: 0.0779s/iter; left time: 170.9067s
Epoch: 2 cost time: 21.098941564559937
Epoch: 2, Steps: 266 | Train Loss: 0.2949652 Vali Loss: 0.6190213 Test Loss: 0.4046888
Validation loss decreased (0.625544 --> 0.619021).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.3215503

## Load Model

In [4]:
from models.Autoformer import Model as Autoformer


model = Autoformer(args)
model.load_state_dict(torch.load('checkpoints/{}/checkpoint.pth'.format(setting)))


<All keys matched successfully>

## Load Data

In [5]:
from data_provider.data_loader import Dataset_Custom

data_set = Dataset_Custom(
    root_path=args.root_path,
    data_path=args.data_path,
    flag="train",
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    target=args.target,
    timeenc= 0 if args.embed != 'timeF' else 1,
    freq=args.freq
)
len(data_set)

12075

In [6]:
seq_x, seq_y, seq_x_mark, seq_y_mark = data_set[0]
seq_x.shape, seq_y.shape, seq_x_mark.shape, seq_y_mark.shape


((96, 7), (72, 7), (96, 4), (72, 4))

In [7]:
from torch.utils.data import DataLoader

data_loader = DataLoader(data_set, batch_size=1, shuffle=True)
len(data_loader)

12075

In [8]:
batch_x , batch_y, batch_x_mark, batch_y_mark = next(iter(data_loader))
batch_x.shape, batch_y.shape, batch_x_mark.shape, batch_y_mark.shape

(torch.Size([1, 96, 7]),
 torch.Size([1, 72, 7]),
 torch.Size([1, 96, 4]),
 torch.Size([1, 72, 4]))

In [9]:
batch_x = batch_x.float()
batch_y = batch_y.float()
batch_x_mark = batch_x_mark.float()
batch_y_mark = batch_y_mark.float()

In [15]:
concat = torch.cat([batch_x, batch_y], dim=1)[:,:,0]
concat.shape

torch.Size([1, 168])

In [17]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

plt.plot(concat[0,:])
plt.savefig('concat.png')

## Prediction

In [ ]:
import os
import matplotlib.pyplot as plt

def visualize_pred(batch_x, batch_y, output, result_dir="img"):
    os.makedirs(result_dir, exist_ok=True)
    batch_x_np = batch_x.cpu().detach().numpy()
    batch_y_np = batch_y.cpu().detach().numpy()
    output_np = output.cpu().detach().numpy()

    seq_len = len(batch_x_np[0, :, 0])      # longueur de la séquence d'entrée
    out_len = len(output_np[0, :, 0])       # longueur de la séquence de sortie

    for i in range(batch_x_np.shape[-1]):
        plt.figure(figsize=(20, 10))

        # Affichage de la séquence d'entrée
        plt.plot(range(seq_len), batch_x_np[0, :, i], label='Input Sequence')

        # Affichage de la séquence de sortie réelle en continuité de la séquence d'entrée
        plt.plot(range(seq_len - 1, seq_len + out_len - 1), batch_y_np[0, -out_len:, i], label='True Output Sequence')

        # Affichage de la séquence de sortie prédite en continuité de la séquence d'entrée
        plt.plot(range(seq_len - 1, seq_len + out_len - 1), output_np[0, :, i], label='Predicted Output Sequence')

        # Mise en forme de la figure
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.savefig(f'{result_dir}/pred_{i}.png')
        plt.close()

## Vanilla

In [22]:

# x_enc, x_mark_enc, x_dec, x_mark_dec
output = model(batch_x, batch_x_mark, batch_y, batch_y_mark)
output.shape

torch.Size([1, 24, 7])

In [23]:
visualize_pred(batch_x, batch_y, output,result_dir="vanilla")

## Dec_inp

In [37]:
dec_inp = torch.zeros_like(batch_x[:, -args.pred_len:, :]).float()
dec_inp = torch.cat([batch_x[:, :args.label_len, :], dec_inp], dim=1).float().to(exp.device)
dec_inp.shape

torch.Size([1, 72, 7])

In [38]:
dec_inp

tensor([[[-0.0754,  0.1832,  0.1326,  0.3960, -1.0694, -0.7178, -1.5979],
         [ 0.2515,  1.0076,  0.4731,  1.4284, -1.0181, -0.6257, -1.5726],
         [ 0.4308,  0.9442,  0.5885,  1.4284, -1.2747, -0.7178, -1.6485],
         [ 0.5363,  0.9442,  0.7443,  1.5945, -1.3260, -0.7646, -1.6064],
         [ 0.3992,  1.1027,  0.6577,  1.5208, -1.3774, -0.7646, -1.3957],
         [ 0.3781,  1.2612,  0.6520,  1.6494, -1.3774, -0.7646, -1.4378],
         [ 0.1882,  0.7539,  0.4615,  1.4098, -1.2234, -0.5804, -1.4884],
         [ 0.3464,  0.9759,  0.5481,  1.5021, -1.1468, -0.6725, -1.4547],
         [ 0.4308,  0.6588,  0.5596,  0.9678, -0.6849, -0.4883, -1.5811],
         [ 0.8632,  1.2295,  1.0272,  1.5389,  0.1859, -0.2121, -1.5726],
         [ 0.6313,  1.2612,  0.7040,  1.2993,  0.1859,  0.1548, -1.3535],
         [ 0.2620,  0.9442,  0.2942,  0.9678,  0.0328,  0.3857, -1.2609],
         [ 0.2831,  0.7539,  0.2942,  0.7649, -0.0186,  0.3389, -1.2356],
         [ 0.5258,  0.8173,  0.5078,  

In [39]:
outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
visualize_pred(batch_x, batch_y, output,result_dir="dec")